In [ ]:
# | default_exp _cli

In [ ]:
# | export

from typing import *
from pathlib import Path
from asyncio import run as aiorun
import importlib
import sys


import typer

from fast_kafka_api.application import FastKafkaAPI

[INFO] fast_kafka_api._components.asyncapi: ok


In [ ]:
import os
from typer.testing import CliRunner
from tempfile import TemporaryDirectory
from contextlib import contextmanager

from nbconvert import PythonExporter
import nbformat

In [ ]:
@contextmanager
def cwd(path: Union[str, Path]) -> None:
    org_cwd = os.getcwd()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(org_cwd)
        
with cwd("/tmp"):
    assert os.getcwd() == "/tmp"
assert os.getcwd() != "/tmp"
os.getcwd()

'/work/fast-kafka-api/nbs'

In [ ]:
def generate_app_src(out_path: Union[Path, str]) -> None:
    path = Path("099_Test_Service.ipynb")
    if not path.exists():
        path = Path("..") / "099_Test_Service.ipynb"
    if not path.exists():
        raise ValueError(f"Path '{path.resolve()}' does not exists.")

    with open(path, "r") as f:
        notebook = nbformat.reads(f.read(), nbformat.NO_CONVERT)
        exporter = PythonExporter()
        source, _ = exporter.from_notebook_node(notebook)
        
    with open(out_path, "w") as f:
        f.write(source)

In [ ]:
with TemporaryDirectory() as d:
    generate_app_src((Path(d) / "main.py"))
    !ls -al {d}
    !cat {d}/main.py | grep @app

total 20
drwx------ 2 davor davor  4096 Jan  5 14:33 .
drwxrwxrwt 1 root  root   4096 Jan  5 14:33 ..
-rw-rw-r-- 1 davor davor 11632 Jan  5 14:33 main.py
    @app.get("/docs", include_in_schema=False)
    @app.get("/redoc", include_in_schema=False)
    @app.post("/from_kafka_start")
    @app.get("/from_kafka_end")
    @app.consumes()  # type: ignore
    @app.consumes()  # type: ignore
    @app.produces()  # type: ignore
    @app.produces()  # type: ignore
    @app.produces()  # type: ignore
    @app.produces()  # type: ignore


In [ ]:
# | export


class ImportFromStringError(Exception):
    pass


def _import_from_string(import_str: str) -> Any:
    """Imports library from string

    Note:
        copied from https://github.com/encode/uvicorn/blob/master/uvicorn/importer.py

    Args:
        import_str: input string in form 'main:app'

    """
    sys.path.append(".")

    if not isinstance(import_str, str):
        return import_str

    module_str, _, attrs_str = import_str.partition(":")
    if not module_str or not attrs_str:
        message = (
            'Import string "{import_str}" must be in format "<module>:<attribute>".'
        )
        typer.secho(f"{message}", err=True, fg=typer.colors.RED)
        raise ImportFromStringError(message.format(import_str=import_str))

    try:
        module = importlib.import_module(module_str)
    except ImportError as exc:
        if exc.name != module_str:
            raise exc from None
        message = 'Could not import module "{module_str}".'
        raise ImportFromStringError(message.format(module_str=module_str))

    instance = module
    try:
        for attr_str in attrs_str.split("."):
            instance = getattr(instance, attr_str)
    except AttributeError:
        message = 'Attribute "{attrs_str}" not found in module "{module_str}".'
        raise ImportFromStringError(
            message.format(attrs_str=attrs_str, module_str=module_str)
        )

    return instance

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with cwd(d):
        app = _import_from_string(f"{src_path.stem}:app")
        assert isinstance(app, FastKafkaAPI)

[INFO] main: check


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")


@_app.command(
    help="Runs Fast Kafka API application using uvicorn",
)
def run(root_path: str = typer.Option(".", help="")):
    try:
        raise NotImplementedError()
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)


@_app.command(
    help="Creates documentation for a Fast Kafka API application ",
)
def generate_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be create"
    ),
    app: str = typer.Argument(..., help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafkaAPI**."),
):
    try:
        application = _import_from_string(app)
        application.generate_async_spec()
    except Exception as e:

        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
result = runner.invoke(_app, ["--help"])

                                                                                                                   
 Usage: root [OPTIONS] COMMAND [ARGS]...                                                                           
                                                                                                                   

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --install-completion          Install completion for the current shell.                                         │
│ --show-completion             Show completion for the current shell, to copy it or customize the installation.  │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Commands ──────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ generate-docs           Creates documentation for a Fast Kafka API application                                  │
│ run                     Runs Fast Kafka API application using uvicorn                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["generate-docs", "--help"])

                                                                                                                   
 Usage: root generate-docs [OPTIONS] APP                                                                           
                                                                                                                   

Creates documentation for a Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafkaAPI**.                                                         │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be create [default: .]                        │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    print(f"{src_path=}")
    with cwd(d):
        !ls -al {d}
        import_str = f"{src_path.stem}:app"

        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0


src_path=PosixPath('/tmp/tmpecuz48zt/main.py')
total 20
drwx------ 2 davor davor  4096 Jan  5 14:33 .
drwxrwxrwt 1 root  root   4096 Jan  5 14:33 ..
-rw-rw-r-- 1 davor davor 11632 Jan  5 14:33 main.py
[INFO] fast_kafka_api._components.asyncapi: Old async specifications at '/tmp/tmpecuz48zt/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fast_kafka_api._components.asyncapi: New async specifications generated at: 'asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fast_kafka_api._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmpecuz48zt/asyncapi/docs.



[INFO] fast_kafka_api._components.asyncapi: Keeping the old async specifications at: 'asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Skipping generating async documentation in '/tmp/t

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: root run [OPTIONS]                                                                                         
                                                                                                                   

Runs Fast Kafka API application using uvicorn

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  [default: .]                                                                           │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["run"])
print(result.output)
assert result.return_value != 0

Unexpected internal error: 

